# 简介

[openai-python](https://github.com/openai/openai-python)是`OpenAI`的官方`Python`库.

# 安装

```bash
pip install openai
```

# 使用

## 基本使用

首先要创建一个客户端对象.

In [8]:
from openai import OpenAI

open_client = OpenAI(
    base_url="https://api.juheai.top/v1/",
    api_key='sk-RLbK5xgSbH1YsoYW8fA55c5b03864108930316F7178bF970'
)



## 文本补全

### 简单对话

In [2]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""你好"""
        }
    ]
)
result_text = response.choices[0].message.content.strip()
print(result_text)

你好！有什么我可以帮助你的吗？


### 返回`json`格式

In [10]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""随机生成以下json格式的数据: 
        {{
            "name": "张三",
            "age": 18,
        }}
"""
        }
    ],
    response_format={"type": "json_object"}
)
# print(response)
result_text = response.choices[0].message.content.strip()
print(result_text)

{
    "name": "李四",
    "age": 25
}
